In [1]:
import pandas as pd
import numpy as np
import os

# read preprocessed data
cwd = os.getcwd()
df = pd.read_csv(cwd + "/data_csv/preprocessing_data.csv")

In [2]:
df

,image,emotion
0,Aaron_Peirsol_0003.jpg,4
1,Aaron_Sorkin_0001.jpg,4
2,Aaron_Sorkin_0002.jpg,4
3,Abbas_Kiarostami_0001.jpg,5
4,Abdel_Madi_Shabneh_0001.jpg,5
...,...,...
4809,Zoran_Djindjic_0001.jpg,5
4810,Zoran_Djindjic_0002.jpg,5
4811,Zorica_Radovic_0001.jpg,5
4812,Zulfiqar_Ahmed_0001.jpg,5


In [3]:
def get_distance(fist_point, second_point):
    distance =  math.sqrt(math.pow(fist_point[0] - second_point[0], 2) + math.pow(fist_point[1] - second_point[1], 2))
    return abs(distance)

In [4]:
import cv2, math, numpy as np, dlib
from imutils import face_utils

# clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(cwd + "/predictor/shape_predictor_68_face_landmarks.dat")

error = []
mlist = []
distlist = []
eye_size_list = []
eye_brows_list = []

# run each row in train.csv
for idx, row in df.iterrows():
    imagePath = cwd + "/cleaned_images/" + row.image
    image = cv2.imread(imagePath)

    rects = detector(image, 0)

    if len(rects) == 0:
        error.append(row.image)

    xlist = []
    ylist = []
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks and convert the facial landmark (x, y)
        shape = predictor(image, rect)
        shape = face_utils.shape_to_np(shape)

        # loop over coordinates, draw them on the image and store coordinates in two lists
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
            xlist.append(x)
            ylist.append(y)

    # get mean of both axes to determine centre of gravity
    xmean = np.mean(xlist) 
    ymean = np.mean(ylist)

    # plot central face on image
    cv2.circle(image, (int(xmean), int(ymean)), 1, (0, 255, 0), -1)

    # find distance between mouth
    mavg = np.mean([ylist[61] - ylist[67], ylist[62] - ylist[66], ylist[63] - ylist[65]])
    mlist.append(mavg)

    # find distance between left eye
    left_eye_avg = np.mean([
        get_distance([xlist[37], ylist[37]], [xlist[40], ylist[40]]),
        get_distance([xlist[38], ylist[38]], [xlist[41], ylist[41]])
    ])

    # find distance between right eye
    right_eye_avg = np.mean([
        get_distance([xlist[43], ylist[43]], [xlist[46], ylist[46]]),
        get_distance([xlist[44], ylist[44]], [xlist[47], ylist[47]])
    ])
    eye_size_list.append(np.mean([left_eye_avg, right_eye_avg]))

    # find distance between eye browns
    eye_brows = np.mean([ylist[24] - ylist[26], ylist[19] - ylist[17]])
    eye_brows_list.append(eye_brows)

    # find distance between every poin to central point
    templist = []
    for i in range(17, 68):
        dist = math.sqrt(math.pow(xlist[i] - xmean, 2) + math.pow(ylist[i] - ymean, 2))
        templist.append(dist)
    distavg = np.mean(dist)
    distlist.append(distavg)
  
    # show the output image with the face detections + facial landmarks
    cv2.imshow("Output", image)# 
    k = cv2.waitKey(5) & 0xFF
    if k == 68:
        break

print(len(error))

0


In [10]:
# add feature to df
df['mouth_distance'] = mlist
df['average_distance'] = distlist
df['eye_size'] = eye_size_list
df['eye_brows'] = eye_brows_list

df.to_csv(cwd + "/data_csv/preprocessing_data.csv", index=False)

In [11]:
df

,image,emotion,mouth_distance,average_distance,eye_size,eye_brows
0,Aaron_Peirsol_0003.jpg,4,-19.000000,66.524459,20.163441,-20.5
1,Aaron_Sorkin_0001.jpg,4,-34.666667,81.816150,21.731924,-16.5
2,Aaron_Sorkin_0002.jpg,4,-28.000000,75.358378,21.251187,-25.5
3,Abbas_Kiarostami_0001.jpg,5,0.000000,51.630016,25.085926,-34.0
4,Abdel_Madi_Shabneh_0001.jpg,5,-3.000000,68.103175,24.153298,-15.0
...,...,...,...,...,...,...
4809,Zoran_Djindjic_0001.jpg,5,-3.000000,59.068719,25.890925,-24.5
4810,Zoran_Djindjic_0002.jpg,5,-19.666667,68.181357,22.351973,-17.5
4811,Zorica_Radovic_0001.jpg,5,-7.333333,78.923287,29.033653,-30.0
4812,Zulfiqar_Ahmed_0001.jpg,5,-0.666667,58.840270,17.860849,-29.0
